# **Preprocessing data**

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool, metrics, cv
import numpy as np 
from numpy.random import Generator
import hyperopt
from imblearn.over_sampling import SMOTE


path = '/kaggle/input/train-data/train_share.tsv'
train = pd.read_table(path)

path = '/kaggle/input/test-data-allcups-f/test_share.tsv'
test = pd.read_table(path)


categorical = ["feature_1", "feature_3", "feature_5", "feature_6", "feature_8", "feature_9", "feature_11", "feature_12", "feature_13", "feature_14", "feature_16", "feature_18", "feature_23", "feature_24", "feature_26", "feature_28", "application_2"]


def data_preprocessing(data):
    for column in categorical:
        data[column] = data[column].fillna(-1)
        data[column] = data[column].astype('str')
    data = data.fillna(0)
    return data


from sklearn.model_selection import train_test_split

test = data_preprocessing(test).drop(columns = 'Unnamed: 0')

train = data_preprocessing(train)
data_train, data_val  = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

y_train = data_train.copy()['target']
X_train = data_train.copy().drop(columns = 'target')
y_val = data_val.copy()['target']
X_val = data_val.copy().drop(columns = 'target')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
num_of_test_data = int(97211*0.2)

data_train_1, data_val_1  = train_test_split(train[train['target'] == 1], test_size=num_of_test_data, random_state=42, shuffle=True)
data_train_0, data_val_0  = train_test_split(train[train['target'] == 0], test_size=num_of_test_data, random_state=42, shuffle=True)

data_train = pd.concat([data_train_0, data_train_1])
data_val = pd.concat([data_val_0, data_val_1])

y_train = data_train.copy()['target']
X_train = data_train.copy().drop(columns = 'target')
y_val = data_val.copy()['target']
X_val = data_val.copy().drop(columns = 'target')

In [ ]:
y_val.value_counts()

In [ ]:
OS = SMOTE(random_state=12) # если нужен смоте
X_train_OS, y_train_OS = OS.fit_resample(X_train, y_train)

def data_preprocessing(data):
    for column in categorical:
        data[column] = data[column].astype('str')
    data = data.fillna(0)
    return data

X_train_OS = data_preprocessing(X_train_OS)

In [ ]:
numeric = list(set(train.columns) - set(categorical))

In [ ]:
train.dtypes

In [ ]:
for column in train.columns:
    print(f'{column}: {train[column].nunique(), train[column].isna().sum()}')

# **AutoML**

In [ ]:
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init(bind_to_localhost=False)

# Загрузка данных
def data_preprocessing(data):
    for column in categorical:
        if column != "target":
            data[column] = [int(num) if not np.isnan(num) else np.nan for num in data[column]]
    return data

categorical = ["feature_1", "feature_3", "feature_5", "feature_6", "feature_8", "feature_9", "feature_11", "feature_12", "feature_13", "feature_14", "feature_16", "feature_18", "feature_23", "feature_24", "feature_26", "feature_28", "application_2", "target"]

path = '/kaggle/input/train-data/train_share.tsv'
train = pd.read_table(path)
train = data_preprocessing(train)

data = h2o.H2OFrame(train)

# перевод данных в катигориалки
for name in categorical:
    data[name] = data[name].asfactor()
data['target'] = data['target'].asfactor()

# перевод данных в нум
numeric = list(set(train.columns) - set(categorical))
for name in numeric:
    if name != "target":
        data[name] = data[name].asnumeric()

y = 'target'
X = data.columns
X.remove(y)

train, valid = data.split_frame(ratios=[0.8])


Automl = H2OAutoML(max_models = 80, seed=1, sort_metric='logloss')
Automl.train(x=X, y=y, training_frame=train, validation_frame=valid)

print(Automl.leader)

h2o.save_model(model=Automl.leader, path="leader_model", force=True)

leaderboard = Automl.leaderboard
print(leaderboard)


top_n = 5  
top_models = leaderboard[:top_n, "model_id"]  

# Сохранение нескольких лучших моделей
for i, model_id in enumerate(top_models.as_data_frame().values.flatten()):
    model = h2o.get_model(model_id)  # Получение модели по model_id
    model_path = h2o.save_model(model=model, path=f"directory/model_{i+1}", force=True)
    print(f"Модель {i+1} сохранена по пути: {model_path}")

path = '/kaggle/input/test-data-allcups-f/test_share.tsv'
test = pd.read_table(path)

test = data_preprocessing(test).drop(columns = 'Unnamed: 0')

test = h2o.H2OFrame(test)

for name in categorical:
    if name != "target":
        test[name] = test[name].asfactor()

numeric = list(set(test.columns) - set(categorical))
for name in numeric:
    if name != "target":
        test[name] = test[name].asnumeric()

preds = Automl.predict(test)

preds = h2o.as_list(preds)

preds.to_csv('Model_pred.csv', index=False)

# **Catboost Accuracy**

In [4]:
scale_pos_weight_value = y_train.value_counts()
scale_pos_weight_value = y_train.value_counts()[0]/y_train.value_counts()[1]

model = CatBoostClassifier(
    scale_pos_weight = scale_pos_weight_value,
    l2_leaf_reg=int(2),
    learning_rate=0.10370814909478171,
    loss_function = 'Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
    task_type="GPU",
    devices='0',
    use_best_model=True
)

model.fit(
    X_train, y_train,
    cat_features=categorical,
    eval_set=(X_val, y_val),
    logging_level='Silent' # Verbose если нужен вывод обучения # Silent если не нужен вывод обучения
)

In [5]:
predict = model.predict_proba(test)[:,1:].flatten()
submission = pd.DataFrame({'id' : test.index, 'target' : predict})
submission.to_csv('submissionAccuracy.csv', index=False)

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        eval_metric=metrics.Accuracy(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
        custom_loss=[metrics.Accuracy()],
        iterations=500,
        task_type="GPU",
        devices='0',
    )

    categorical_features_indices = [0, 1, 2, 3, 4, 5, 7, 8]

    cv_data = cv(
        Pool(X_val, y_val, cat_features=categorical),
        model.get_params(),
        logging_level='Silent',
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])

    return 1 - best_accuracy

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print(best)

# **CatBoost BalancedAccuracy**

In [ ]:
model = CatBoostClassifier(
    auto_class_weights='SqrtBalanced',
    l2_leaf_reg=int(4.0),
    learning_rate=0.13708243344030624,
    min_data_in_leaf = int(76.0),
    depth = int(6.0),
    loss_function='Logloss',
    eval_metric='BalancedAccuracy',
    custom_loss=[metrics.BalancedAccuracy()],
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
    task_type="GPU",
    devices='0',
    use_best_model=True
)

model.fit(
    X_train, y_train,
    cat_features=categorical,
    eval_set=(X_val, y_val),
    logging_level='Verbose' # Verbose если нужен вывод обучения # Silent если не нужен вывод обучения
)

In [ ]:
predict = model.predict_proba(test)[:,1:].flatten()
submission = pd.DataFrame({'id' : test.index, 'target' : predict})
submission.to_csv('submission.csv', index=False)

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        scale_pos_weight=scale_pos_weight_value,
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        eval_metric=metrics.Recall(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
        custom_loss=[metrics.Recall()],
        iterations=500,
        task_type="GPU",
        devices='0',
    )

    cv_data = cv(
        Pool(X_val, y_val, cat_features=categorical),
        model.get_params(),
        logging_level='Silent',
    )
    best_recall = np.max(cv_data['test-Recall-mean'])

    return 1 - best_recall

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 5, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 3e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print(best)

# **Catboost Recall**

In [ ]:
class RecallatK:
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, targets, weight):
        threshold = int(len(targets)*0.15)
        
        top_k_preds = np.argsort(approxes[0])[-threshold:]
        
        total_positives = np.sum(targets == 1)
        
        error = np.sum(targets[top_k_preds])/total_positives
        
        output = (error, np.sum(weight))    
        
        return output

In [ ]:
scale_pos_weight_value = y_train.value_counts()
scale_pos_weight_value = y_train.value_counts()[0]/y_train.value_counts()[1]

model = CatBoostClassifier(
   # scale_pos_weight = scale_pos_weight_value,
   # l2_leaf_reg=int(2),
   # learning_rate=0.10370814909478171, # -------------------------------------------------------------------------------
    loss_function = 'Logloss',
    eval_metric=RecallatK(),
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
  #  task_type="GPU",
   # devices='0',
    use_best_model=True
)

model.fit(
    X_train, y_train,
    cat_features=categorical,
    eval_set=(X_val, y_val),
    logging_level='Verbose' # Verbose если нужен вывод обучения # Silent если не нужен вывод обучения
)

In [ ]:
predict = model.predict_proba(test)[:,1:].flatten()
submission = pd.DataFrame({'id' : test.index, 'target' : predict})
submission.to_csv('submission_F_Recall.csv', index=False)

In [ ]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        scale_pos_weight=scale_pos_weight_value,
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        eval_metric=metrics.Recall(),
        random_seed=42,
        verbose=False,
        loss_function=metrics.Logloss(),
        custom_loss=[metrics.Recall()],
        iterations=500,
        task_type="GPU",
        devices='0',
    )

    cv_data = cv(
        Pool(X_val, y_val, cat_features=categorical),
        model.get_params(),
        logging_level='Silent',
    )
    best_recall = np.max(cv_data['test-Recall-mean'])

    return 1 - best_recall

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 5, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 3e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print(best)

# **Catboost param podbor**

In [7]:
import pandas as pd
import cupy as cp
from catboost import CatBoostClassifier, Pool, metrics, cv
import numpy as np
from sklearn.model_selection import train_test_split
import hyperopt
print('start')

path = '/kaggle/input/train-data/train_share.tsv'
train = pd.read_table(path)

path = '/kaggle/input/test-data-allcups-f/test_share.tsv'
test = pd.read_table(path)
print('Data dowloaded')
categorical = ["feature_1", "feature_3", "feature_5", "feature_6", "feature_8", "feature_9", "feature_11", "feature_12", "feature_13", "feature_14", "feature_16", "feature_18", "feature_23", "feature_24", "feature_26", "feature_28", "application_2"]

def data_preprocessing(data):
    for column in categorical:
        if column != "target":
            data[column] = data[column].fillna(-1)
            data[column] = data[column].astype('str')

    numeric = list(set(train.columns) - set(categorical))
    for name in numeric:
        if name != "target":
            data[name] = data[name].fillna(0.0)

    return data

test = data_preprocessing(test).drop(columns = 'Unnamed: 0')

train = data_preprocessing(train)

data_train, data_val  = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

y_train = data_train['target']
X_train = data_train.drop(columns = 'target')
y_val = data_val['target']
X_val = data_val.drop(columns = 'target')

class RecallatK:
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True
    
    def evaluate(self, approxes, targets, weight):   
        threshold = int(len(targets) * 0.15)
        
        top_k_preds = np.argsort(approxes[0])[-threshold:]

        total_positives = np.sum(targets == 1)

        error = np.sum(targets[top_k_preds]) / total_positives

        output = (error, np.sum(weight))

        return output


def hyperopt_objective(params):
    model = CatBoostClassifier(
        auto_class_weights='SqrtBalanced',
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
       # depth=int(params['depth']),
       # subsample=params['subsample'],
       # colsample_bylevel=params['colsample_bylevel'],
        min_data_in_leaf=int(params['min_data_in_leaf']),
        eval_metric=RecallatK(),
      #  eval_metric = 'Logloss', 
        random_seed=42,
        verbose=False,
        loss_function='Logloss',
     #   custom_loss=[metrics.Logloss()],
        iterations=1000,
        task_type="GPU",
        devices='0',
    )
    cv_data = cv(
        Pool(X_val, y_val, cat_features=categorical),
        model.get_params(),
        logging_level='Verbose',
    )
    best_recall = np.max(cv_data['test-RecallatK-mean'])

    return 1 - best_recall


from numpy.random import Generator

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 3e-1),
    'depth': hyperopt.hp.choice('depth', [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0]),
  #  'subsample': hyperopt.hp.uniform('subsample', 0.05, 1.0),
  #  'colsample_bylevel': hyperopt.hp.uniform('colsample_bylevel', 0.05, 1.0),
    'min_data_in_leaf': hyperopt.hp.quniform('min_data_in_leaf', 1, 100, 1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print(best)

start
Data dowloaded
Training on fold [0/3]                                

  0%|          | 0/50 [00:01<?, ?trial/s, best loss=?]

Default metric period is 5 because PythonUserDefinedPerObject
 is/are not implemented for GPU



bestTest = 0.436985237                                

bestIteration = 315                                   

Training on fold [1/3]                                

bestTest = 0.4414918415                               

bestIteration = 345                                   

Training on fold [2/3]                                

bestTest = 0.4315462315                               

bestIteration = 65                                    

Training on fold [0/3]                                                             

  2%|▏         | 1/50 [04:39<3:47:11, 278.19s/trial, best loss: 0.5654493654493655]

Default metric period is 5 because PythonUserDefinedPerObject
 is/are not implemented for GPU



bestTest = 0.4372960373                                                            

bestIteration = 750                                                                

Training on fold [1/3]                                                             

bestTest = 0.4416472416                                                            

bestIteration = 360                                                                

Training on fold [2/3]                                                             

bestTest = 0.4315462315                                                            

bestIteration = 945                                                                

Training on fold [0/3]                                                             

  4%|▍         | 2/50 [09:17<3:42:26, 278.06s/trial, best loss: 0.5647759647759647]

...

bestTest = 0.4388500389                                                             

bestIteration = 930                                                                 

Training on fold [1/3]                                                              

bestTest = 0.4396270396                                                             

bestIteration = 990                                                                 

Training on fold [2/3]                                                              

bestTest = 0.4357420357                                                             

bestIteration = 350                                                                 

Training on fold [0/3]                                                              

 96%|█████████▌| 48/50 [3:40:35<09:21, 280.60s/trial, best loss: 0.5598031598031599]

Default metric period is 5 because PythonUserDefinedPerObject
 is/are not implemented for GPU



bestTest = 0.4146076146                                                             

bestIteration = 930                                                                 

Training on fold [1/3]                                                              

bestTest = 0.4208236208                                                             

bestIteration = 970                                                                 

Training on fold [2/3]                                                              

bestTest = 0.4163170163                                                             

bestIteration = 995                                                                 

Training on fold [0/3]                                                              

 98%|█████████▊| 49/50 [3:45:35<04:46, 286.40s/trial, best loss: 0.5598031598031599]

Default metric period is 5 because PythonUserDefinedPerObject
 is/are not implemented for GPU



bestTest = 0.4346542347                                                             

bestIteration = 970                                                                 

Training on fold [1/3]                                                              

bestTest = 0.4405594406                                                             

bestIteration = 835                                                                 

Training on fold [2/3]                                                              

bestTest = 0.434032634                                                              

bestIteration = 735                                                                 

100%|██████████| 50/50 [3:50:17<00:00, 276.36s/trial, best loss: 0.5598031598031599]
{'depth': 4, 'l2_leaf_reg': 4.0, 'learning_rate': 0.05559745248224224, 'min_data_in_leaf': 67.0}
